In [3]:
#get the imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import inspect
import random
import os

In [18]:
#return keypoints from an image using SURF
def getSURFMatches(image,param):

    #use SURF to get set of keypoints and corresponding descriptors
    sift = cv2.xfeatures2d.SURF_create(param)
    kp1,des1 = sift.detectAndCompute(image,None)
    return kp1,des1

#returns set of keypoints from the set of images provided
def getRefSet(imageSet,param):
    
    pointsSet = []
    for img in imageSet:
        p,d = getSURFMatches(img,param)
        pointsSet += [d]
    return pointsSet

#returns a set of images present in a folder
def getImageSet(foldername):
    images = [cv2.imread(foldername + str(i)) for i in os.listdir(foldername)]
    return images

#computes the set of points in a query image that are less than an average distance from 
#the set of reference points
def getBestPoints(rpset,qpset):
    
    dists = []
    for q in qpset:
        sumn = np.sum(np.linalg.norm(rpset - q))
        dists += [sumn]
    
    avg = sum(dists)/len(dists)
    points = []
    for i,d in enumerate(dists):
        if d < avg:
            points += [qpset[i]]
            
    return points

#computes the set of stable points for as set of images using n-1 cross validation methods
def createTestSet(imageSet,param):
    
    #get total set of SURF points for all images
    totalPset = getRefSet(imageSet,param)
    tsetPoints = []
    for i,qpset in enumerate(totalPset):
        #reference set
        rpset = np.concatenate(totalPset[0:i] + totalPset[i+1:])
        tsetPoints += [getBestPoints(rpset,qpset)]
        
    return np.concatenate(tsetPoints)
        

In [118]:
def get_distance(image,stable_points,theta):
    img = cv2.imread(image)
    kp,ds = getSURFMatches(img,400)
    good = 0
    bad = 0
    for i in range(ds.shape[0]):
        sumn = (np.sum((stable_points - ds[i])**2,axis = 1))**(0.5)
#         print (min(sumn),max(sumn))
        if (min(sumn) <= theta):
            good += 1
        else:
            bad += 1
    
    return good,bad

In [133]:
def get_theta(stable_keypoint):
    genuine_pics = glob.glob('../Dataset_4NSigComp2010/TrainingSet/Genuine/*.png')
    forged_pic = glob.glob('../Dataset_4NSigComp2010/TrainingSet/Simulated/*.png')
    disguised_pic = glob.glob('../Dataset_4NSigComp2010/TrainingSet/Disguise/*.png')
    theta = 0.17
    tp = 0
    fn = 0
    fp = 0
    tn = 0
    print (len(genuine_pics),len(forged_pic),len(disguised_pic))
    for i in range(len(genuine_pics)):
        good_points,bad_points = get_distance(genuine_pics[i],stable_keypoint,theta)
        print (i,"Good Points",good_points,"Bad Points",bad_points,good_points/(good_points + bad_points))
#         if i == 5:
#             break
        
    print ("\n")
    for i in range(len(forged_pic)):
        good_points,bad_points = get_distance(forged_pic[i],stable_keypoint,theta)
        print (i,"Good Points",good_points,"Bad Points",bad_points,good_points/(good_points + bad_points))
#         if i == 5:
#             break
    
    print ("\n")
    for i in range(len(disguised_pic)):
        good_points,bad_points = get_distance(disguised_pic[i],stable_keypoint,theta)
        print (i,"Good Points",good_points,"Bad Points",bad_points,good_points/(good_points + bad_points))
#         if i == 5:
#             break
    
#     FPR = fp/(fp + tn)
#     FNR = fn/(tp + fn)
#     print (tp,tn,fp,fn)
    

In [131]:
images = getImageSet("/home/shad3/Downloads/Computer_vision/Project/Dataset_4NSigComp2010/TrainingSet/Reference/")

In [84]:
test_set = createTestSet(images,400)

In [134]:
get_theta(test_set)

76 104 20
0 Good Points 1205 Bad Points 714 0.627931214174049
1 Good Points 1379 Bad Points 1033 0.5717247097844113
2 Good Points 1500 Bad Points 1068 0.5841121495327103
3 Good Points 1444 Bad Points 1105 0.5664966653589643
4 Good Points 1454 Bad Points 1150 0.5583717357910907
5 Good Points 1394 Bad Points 983 0.5864535128313
6 Good Points 1565 Bad Points 1032 0.6026184058529072
7 Good Points 1321 Bad Points 1124 0.5402862985685072
8 Good Points 1671 Bad Points 1181 0.5859046283309958
9 Good Points 1681 Bad Points 1154 0.5929453262786596
10 Good Points 1394 Bad Points 1256 0.5260377358490566
11 Good Points 1302 Bad Points 1087 0.5449979070740896
12 Good Points 1642 Bad Points 1154 0.5872675250357654
13 Good Points 1900 Bad Points 1058 0.6423258958755916
14 Good Points 1502 Bad Points 1150 0.5663650075414781
15 Good Points 988 Bad Points 1022 0.49154228855721394
16 Good Points 1582 Bad Points 804 0.6630343671416596
17 Good Points 1496 Bad Points 1147 0.5660234581914491
18 Good Points 12

76 Good Points 1185 Bad Points 1370 0.4637964774951076
77 Good Points 783 Bad Points 816 0.4896810506566604
78 Good Points 990 Bad Points 864 0.5339805825242718
79 Good Points 1248 Bad Points 775 0.6169055857637172
80 Good Points 586 Bad Points 825 0.4153082919914954
81 Good Points 1374 Bad Points 1465 0.4839732300105671
82 Good Points 990 Bad Points 1117 0.4698623635500712
83 Good Points 1076 Bad Points 1201 0.4725516029863856
84 Good Points 1034 Bad Points 1195 0.4638851502916106
85 Good Points 415 Bad Points 721 0.3653169014084507
86 Good Points 1666 Bad Points 1349 0.5525704809286899
87 Good Points 787 Bad Points 1109 0.415084388185654
88 Good Points 1446 Bad Points 1293 0.5279299014238773
89 Good Points 1275 Bad Points 1270 0.5009823182711198
90 Good Points 432 Bad Points 683 0.3874439461883408
91 Good Points 1794 Bad Points 996 0.6430107526881721
92 Good Points 1028 Bad Points 745 0.5798082346305696
93 Good Points 718 Bad Points 731 0.49551414768806074
94 Good Points 851 Bad Poin